In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.metrics import classification_report, accuracy_score

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))


2024-07-22 00:33:30.318123: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-22 00:33:30.332101: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-22 00:33:30.332122: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-22 00:33:30.342477: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-22 00:33:31.009096: W tensorflow/compiler/tf

Num GPUs Available:  1


2024-07-22 00:33:31.854302: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-22 00:33:31.888444: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-22 00:33:31.891595: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [2]:
images_train = np.load("../data/images/images_train.npy")
images_val = np.load("../data/images/images_val.npy")
images_test = np.load("../data/images/images_test.npy")

hog_train = np.load("../data/hog/hog_train.npy")
hog_train = hog_train * 255
hog_val = np.load("../data/hog/hog_val.npy")
hog_val = hog_val * 255
hog_test = np.load("../data/hog/hog_test.npy")
hog_test = hog_test * 255

canny_edges_train  = np.load("../data/canny_edges/canny_edges_train.npy")
canny_edges_val  = np.load("../data/canny_edges/canny_edges_val.npy")
canny_edges_test  = np.load("../data/canny_edges/canny_edges_test.npy")

contours_train = np.load("../data/contours/contours_train.npy")
contours_val = np.load("../data/contours/contours_val.npy")
contours_test = np.load("../data/contours/contours_test.npy")


In [3]:
X_train = np.stack([images_train, hog_train, canny_edges_train, contours_train], axis=-1)
X_train = X_train / 255.0
X_val = np.stack([images_val, hog_val, canny_edges_val, contours_val], axis=-1)
X_val = X_val / 255.0
X_test = np.stack([images_test, hog_test, canny_edges_test, contours_test], axis=-1)
X_test = X_test / 255.0

y_train = np.load("../data/images/label_ids_train.npy")
y_val = np.load("../data/images/label_ids_val.npy")
y_test = np.load("../data/images/label_ids_test.npy")


In [4]:
labels_train = np.load("../data/images/labels_train.npy")
class_lookup = list(set(zip(y_train, labels_train)))
class_lookup = {id_value: class_value for id_value, class_value in class_lookup}
class_lookup


{9: 'avulsion',
 8: 'impacted',
 4: 'dislocation',
 1: 'spiral',
 7: 'oblique',
 6: 'longitudinal',
 0: 'hairline',
 5: 'pathological',
 2: 'greenstick',
 3: 'comminuted'}

In [5]:
input = layers.Input(shape=(X_train.shape[1], X_train.shape[2], X_train.shape[3]))
x = layers.Conv2D(32, (3, 3), activation='relu')(input)
x = layers.MaxPooling2D((2, 2))(x)
x = layers.Conv2D(64, (3, 3), activation='relu')(x)
x = layers.MaxPooling2D((2, 2))(x)
x = layers.Conv2D(64, (3, 3), activation='relu')(x)
x = layers.Flatten()(x)

x = layers.Dense(128, activation='relu')(x)
output = layers.Dense(10, activation='softmax')(x)

model = models.Model(inputs=input, outputs=output)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()


2024-07-22 00:33:35.550155: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-22 00:33:35.559549: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-22 00:33:35.566202: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 512, 512, 4)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 510, 510, 32)   │         1,184 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 255, 255, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 253, 253, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 126, 126, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 124, 124, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 984064)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │   125,960,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 126,018,218 (480.72 MB)

 Trainable params: 126,018,218 (480.72 MB)

 Non-trainable params: 0 (0.00 B)

In [6]:
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=32)


2024-07-22 00:33:36.877248: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 3317694464 exceeds 10% of free system memory.
2024-07-22 00:33:39.812622: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 3317694464 exceeds 10% of free system memory.


Epoch 1/10


I0000 00:00:1721622822.726292 1055228 service.cc:145] XLA service 0x7ccc34005cc0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1721622822.726361 1055228 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce RTX 3080, Compute Capability 8.6
2024-07-22 00:33:42.776864: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-07-22 00:33:42.920492: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8902
2024-07-22 00:33:50.638526: E external/local_xla/xla/service/slow_operation_alarm.cc:65] Trying algorithm eng57{k2=1,k13=0,k14=3,k18=1,k23=0} for conv (f32[32,4,3,3]{3,2,1,0}, u8[0]{0}) custom-call(f32[32,4,512,512]{3,2,1,0}, f32[32,32,510,510]{3,2,1,0}), window={size=3x3}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBackwardFilter", backend_config={"operation_queue_id":"0","wait

 2/25 ━━━━━━━━━━━━━━━━━━━━ 1s 74ms/step - accuracy: 0.0938 - loss: 11.1352

I0000 00:00:1721622834.347036 1055228 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


25/25 ━━━━━━━━━━━━━━━━━━━━ 26s 541ms/step - accuracy: 0.1254 - loss: 9.4062 - val_accuracy: 0.1566 - val_loss: 2.2847
Epoch 2/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 89ms/step - accuracy: 0.2395 - loss: 2.1573 - val_accuracy: 0.2677 - val_loss: 2.1821
Epoch 3/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 86ms/step - accuracy: 0.6676 - loss: 1.1924 - val_accuracy: 0.2626 - val_loss: 2.5366
Epoch 4/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 87ms/step - accuracy: 0.9486 - loss: 0.2313 - val_accuracy: 0.2576 - val_loss: 2.9717
Epoch 5/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 87ms/step - accuracy: 0.9921 - loss: 0.0605 - val_accuracy: 0.2727 - val_loss: 3.4211
Epoch 6/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 91ms/step - accuracy: 0.9943 - loss: 0.0458 - val_accuracy: 0.2778 - val_loss: 3.5795
Epoch 7/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 89ms/step - accuracy: 0.9975 - loss: 0.0160 - val_accuracy: 0.2929 - val_loss: 3.9587
Epoch 8/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 86ms/step - accuracy: 0.9962 - loss: 0.0245 - val_accuracy: 0.2727 - val_loss: 

In [7]:
y_pred = model.predict(X_test)
y_pred = np.argmax(y_pred, axis=1)

accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred, output_dict=True)
report = pd.DataFrame(report).transpose()
report.index = report.index.map(lambda x: class_lookup[int(x)] if x.isdigit() else x)

print(f"Accuracy: {accuracy}")
print("Classification Report:")
report


5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 253ms/step
Accuracy: 0.22857142857142856
Classification Report:


,precision,recall,f1-score,support
hairline,0.166667,0.200000,0.181818,10.000000
spiral,0.166667,0.083333,0.111111,12.000000
greenstick,0.263158,0.312500,0.285714,16.000000
comminuted,0.300000,0.428571,0.352941,14.000000
dislocation,0.166667,0.263158,0.204082,19.000000
pathological,0.181818,0.111111,0.137931,18.000000
longitudinal,0.400000,0.166667,0.235294,12.000000
oblique,0.333333,0.312500,0.322581,16.000000
impacted,0.142857,0.111111,0.125000,9.000000
avulsion,0.200000,0.214286,0.206897,14.000000
